# import libraries

In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.stattools import adfuller

# Data Loading 

In [3]:
tickers = ['TSLA', 'BND', 'SPY']
start_date = '2015-07-01'
end_date = '2025-07-31'

# Fetch the raw data from yfinance
try:
    raw_data = yf.download(tickers, start=start_date, end=end_date)
    print('Data fetched successfully!')
except Exception as e:
    print(f'Error fetching data: {e}')
    raise

# Save the raw DataFrame to CSV for reproducibility
raw_data.to_csv('financial_data.csv')

# Read CSV trying different header depths (3 -> 2 -> 1)
df = None
for header_try in ([0,1,2], [0,1], [0]):
    try:
        df = pd.read_csv('financial_data.csv', index_col=0, parse_dates=True, header=header_try)
        print(f'Read CSV with header={header_try}')
        break
    except Exception:
        df = None

if df is None:
    # last resort: read without headers
    print('Falling back to read without headers')
    df = pd.read_csv('financial_data.csv', header=None)

# helper
prefs = ['adj close', 'adj', 'adjusted', 'close', 'price']
def flatten_col(col):
    if isinstance(col, tuple):
        parts = [str(x) for x in col if (x is not None and str(x).strip() != '')]
        return ' '.join(parts).lower()
    return str(col).lower()

# Extract adj-like columns
adj = None
if isinstance(df.columns, pd.MultiIndex):
    # Try to find columns where any level contains a price-like token
    candidates = [col for col in df.columns if any(tok in flatten_col(col) for tok in prefs)]
    if candidates:
        adj = df[candidates]
    else:
        # Per-ticker extraction: find the column tuple for each ticker
        ticker_map = {}
        for t in tickers:
            matches = [col for col in df.columns if any(str(x).upper() == t.upper() for x in col)]
            if matches:
                # prefer a match that also contains a price-like token
                chosen = None
                for m in matches:
                    if any(tok in flatten_col(m) for tok in ['close','price','adj']):
                        chosen = m
                        break
                if chosen is None:
                    chosen = matches[0]
                ticker_map[t.upper()] = chosen
        if ticker_map:
            adj = df[list(ticker_map.values())]
            # rename columns to tickers
            adj.columns = [k for k in ticker_map.keys()]
        else:
            # As a last fallback, try selecting columns whose top-level is in common price fields
            top_level = [str(x).lower() for x in df.columns.levels[0]]
            fallback_tokens = ['close', 'price']
            if any(tok in ' '.join(top_level) for tok in fallback_tokens):
                candidates = [col for col in df.columns if str(col[0]).lower() in fallback_tokens]
                if candidates:
                    adj = df[candidates]

    if adj is None:
        raise KeyError("Couldn't locate 'Adj Close' or any close-like field in the downloaded data columns")
else:
    # single-level headers
    cols = list(df.columns)
    candidates = [c for c in cols if any(tok in str(c).lower() for tok in prefs)]
    if candidates:
        adj = df[candidates]
    else:
        # maybe columns are tickers already
        if any(str(c).upper() in [t.upper() for t in tickers] for c in cols):
            adj = df[[c for c in cols if str(c).upper() in [t.upper() for t in tickers]]]
        else:
            # give up and take full df
            adj = df

# Normalize resulting object
if isinstance(adj, pd.Series):
    adj = adj.to_frame()

adj.columns = [str(c).upper() for c in adj.columns]
requested = [t.upper() for t in tickers]
available = [c for c in requested if c in adj.columns]
# If available is empty but adj has columns, keep adj as-is
if not available and len(adj.columns) > 0:
    available = list(adj.columns)

adj = adj.reindex(columns=available)
adj.to_csv('adj_close.csv')

data = adj
print('Adj Close extraction complete. Columns:', list(data.columns))

/tmp/ipykernel_54495/3213831329.py:7: FutureWarning: YF.download() has changed argument auto_adjust default to True
  raw_data = yf.download(tickers, start=start_date, end=end_date)
Failed to get ticker 'SPY' reason: Failed to perform, curl: (60) SSL certificate problem: certificate is not yet valid. See https://curl.se/libcurl/c/libcurl-errors.html first for more details.
Failed to get ticker 'BND' reason: Failed to perform, curl: (60) SSL certificate problem: certificate is not yet valid. See https://curl.se/libcurl/c/libcurl-errors.html first for more details.
[                       0%                       ]Failed to get ticker 'TSLA' reason: Failed to perform, curl: (60) SSL certificate problem: certificate is not yet valid. See https://curl.se/libcurl/c/libcurl-errors.html first for more details.
[*********************100%***********************]  2 of 3 completed

3 Failed downloads:
['SPY', 'BND', 'TSLA']: YFTzMissingError('possibly delisted; no timezone found')


Data fetched successfully!
Read CSV with header=[0, 1, 2]
Adj Close extraction complete. Columns: ["('ADJ CLOSE', 'BND', 'UNNAMED: 1_LEVEL_2')", "('ADJ CLOSE', 'SPY', 'UNNAMED: 2_LEVEL_2')", "('ADJ CLOSE', 'TSLA', 'UNNAMED: 3_LEVEL_2')", "('CLOSE', 'BND', 'UNNAMED: 4_LEVEL_2')", "('CLOSE', 'SPY', 'UNNAMED: 5_LEVEL_2')", "('CLOSE', 'TSLA', 'UNNAMED: 6_LEVEL_2')"]
